In [1]:
import os
from src.dataPipeline import DataPipeline

os.chdir('..')

dp = DataPipeline()
df = dp.runPipeline(normalizeAndStandardize=True)

C:\FHNW_Programmiersachen\5_Sem\immo_challenge\src\dataPipeline.py:38: DtypeWarning: Columns (3,4,5,6,11,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,45,46,47,49,50,107,110,114,115,116,119,120,121,124,125,126,128,131,132) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data = pd.read_csv(filePath)
C:\FHNW_Programmiersachen\5_Sem\immo_challenge\src\dataPipeline.py:61: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ]].bfill(axis=1)['Space extracted']
C:\FHNW_Programmiersachen\5_Sem\immo_challenge\src\dataPipeline.py:66: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To o

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor


import torch
from torch.utils.data import Dataset, DataLoader, random_split


# Custom Dataset class
class DataFrameDataset(Dataset):
    def __init__(self, dataframe, feature_columns, label_column):
        self.dataframe = dataframe
        self.features = dataframe[feature_columns].values
        self.labels = dataframe[label_column].values

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        x = torch.tensor(self.features[idx], dtype=torch.float32)
        y = torch.tensor(self.labels[idx], dtype=torch.float32)
        return x, y
    

# Parameters
feature_columns = [col for col in df.columns if col != 'price_cleaned']
label_column = 'price_cleaned'

# Create dataset and dataloader
dataset = DataFrameDataset(df, feature_columns, label_column)

# Set your desired split ratios
train_ratio = 0.7
val_ratio = 0.0
test_ratio = 0.3

# Calculate lengths for each split
total_size = len(dataset)
train_size = int(total_size * train_ratio)
val_size = int(total_size * val_ratio)
test_size = total_size - train_size - val_size  # Ensures all samples are used

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)




In [11]:
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
import wandb
import time
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
import pandas as pd
from IPython.display import IFrame, display



class HelperClass:
    def __init__(self, model, criterion, optimizer):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
        self.wandb_run = None

    def train(self, train_loader,test_loader, num_epochs):
        self.model.to(self.device)
        for epoch in range(num_epochs):
            running_loss = 0.0
            start_traintime = time.time()
            self.model.train()
            for i, (data) in enumerate(train_loader, 0):
                inputs, labels = data
                
                inputs, labels = inputs.to(self.device), labels.to(self.device)

                outputs = self.model(inputs)
                outputs = outputs.squeeze()
                loss = self.criterion(outputs.flatten(), labels.flatten())
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()                
                
                running_loss += loss.item()
            train_loss = running_loss / (i+1)
            endtime_train = time.time() - start_traintime
            test_accuracy, train_accuracy, train_loss, test_loss = self.evaluate(test_loader=test_loader, train_loader=train_loader, train_loss=train_loss, endtime_train=endtime_train)
            print(f"Epoch {epoch+1}, Loss: {train_loss}, Train Accuracy: {train_accuracy}, Test Accuracy: {test_accuracy}")
        print("Finished Training")
        wandb.finish()
        return self.model


    def evaluate(self, test_loader, train_loader, train_loss, endtime_train):
        # Evaluate the model on test_loader
        self.model.to(self.device)
        self.model.eval()

        batches = 0
        test_loss = 0
        test_mape = 0
        starttime_test = time.time()
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                outputs = self.model(inputs)
                outputs = outputs.squeeze()
                test_loss += self.criterion(outputs, labels).item()
                test_mape += mean_absolute_percentage_error(labels, outputs).item()
                batches += 1

        # Calculate average loss and MAPE over all batches
        avg_test_loss = test_loss / batches
        avg_test_mape = test_mape / batches


        batches = 0
        train_mape = 0
        with torch.no_grad():
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                outputs = self.model(inputs)
                outputs = outputs.squeeze()
                train_mape += mean_absolute_percentage_error(labels, outputs).item()
                batches += 1
        
        train_mape = train_mape / batches
        
        endtime_test = time.time() - starttime_test
        wandb.log(
            {
                "test_mape": avg_test_mape,
                "train_mape": train_mape,
                "train_loss": train_loss,
                "test_loss": avg_test_loss,
                "time_train": endtime_train,
                "time_test": endtime_test,

            }    
        )
        return avg_test_mape, train_mape, train_loss, test_loss


def wandb_login(dict, name = None):
    
    # start a new wandb run to track this script
    wandb.init(
        # set the wandb project where this run will be logged
        project="Immo-Challenge",
        name = name,
        # track hyperparameters and run metadata
        config=dict
    )
    
def mean_absolute_percentage_error(preds, targets):
    epsilon = 1e-10  # Small value to avoid division by zero
    return torch.mean(torch.abs((targets - preds) / (targets + epsilon))) * 100
    
def get_run_hist(run_id):
    api = wandb.Api()
    run = api.run(path=f'Immo-Challenge/{run_id}')
    history = run.history()
    return history

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [12]:
def test_model(name: str, linear_layers: int, conv_layers: int):
    learning_rates = [0.00001]
    epochs = 500

    for lr in learning_rates:
        fullyConnectedModel = FullyConnectedModel()
        # Loss function
        criterion = nn.MSELoss()

        # Optimizer
        optimizer = optim.SGD(fullyConnectedModel.parameters(), lr=lr)

        model_class = HelperClass(fullyConnectedModel, criterion, optimizer)

        
        
        dict = {
            "dataset": "CIFAR-10-Normalized",
            "epochs": epochs,
            "linear_layers": linear_layers,
            "learning_rate": lr,
            "architecture": "CNN",
            "batch_size": 32,
            "conv_layers": conv_layers,
        }

        wandb_login(dict, name=f'CNN-Komplexität-{name}-lr{lr}')

        trained_model = model_class.train(train_loader, test_loader, epochs)
        return trained_model

In [13]:
class FullyConnectedModel(nn.Module):
    def __init__(self):
        super(FullyConnectedModel, self).__init__()
        self.fc1 = nn.Linear(109, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 1)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


trained_model = test_model("FullyConnectedModel", 3, 0)

test_loss,█▇▆▅▄▃▂▁
test_mape,▁▁▁▁█▁▂▁
time_test,▇▄▁█▅▆▄▂
time_train,▇▃▁█▃▅▂▃
train_loss,█▇▆▅▄▃▂▁
train_mape,▁▁▄▂██▂▇
test_loss,0.83708
test_mape,86.3086
time_test,0.71702
time_train,0.90914
train_loss,1.16984


Epoch 1, Loss: 1.240620796436228, Train Accuracy: 102.71235521512689, Test Accuracy: 95.5041805189483
Epoch 2, Loss: 1.2325157917693617, Train Accuracy: 107.992464411963, Test Accuracy: 95.56263472109424
Epoch 3, Loss: 1.2245824666004994, Train Accuracy: 113.82456724805435, Test Accuracy: 95.90549387250628
Epoch 4, Loss: 1.2168330516527597, Train Accuracy: 119.91239586961609, Test Accuracy: 96.71853719438825
Epoch 5, Loss: 1.209225345639353, Train Accuracy: 191.71360628557935, Test Accuracy: 98.69069730019083
Epoch 6, Loss: 1.2017681444597714, Train Accuracy: 157.88529637710272, Test Accuracy: 107.28518972591478
Epoch 7, Loss: 1.1944994800428592, Train Accuracy: 128.2380321677978, Test Accuracy: 117.13334247044155
Epoch 8, Loss: 1.1874258502078787, Train Accuracy: 520.0602314654794, Test Accuracy: 101.25667673227738
Epoch 9, Loss: 1.1802983554304298, Train Accuracy: 117.75761067997705, Test Accuracy: 102.28796238801917
Epoch 10, Loss: 1.1731734658780975, Train Accuracy: 111.24226706607

test_loss,███▇▇▆▄▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_mape,▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁█▁▁▁▂▂▁▂▁▁▁▁▁▁
time_test,▅▃█▆▃▃▂▁▁▅▇▂▂▁▁▂▂▅▂▂▂▃▃▃▂▃▄▂▂▂▄▃▃▃▂▂▃▂▂▂
time_train,▃▃▂▄▆▃▄█▁▁▂▂▃▂▆▃▂▄▂▃▁▄▃▃▂▂▂▂▂▂▂▅▃▂▂▃▃▂▃▂
train_loss,█▇▇▇▆▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_mape,▁▇▅▂▃▃▅▃█▄▃▂▃▃▂▂▃▄▃▃▂▂▂▂▂▂▂▂▂▁▂▂▂▁▂▃▄▂▂▂
test_loss,0.12557
test_mape,99.20889
time_test,0.71234
time_train,0.88318
train_loss,0.17954


In [1]:
import os
from src.dataPipeline import DataPipeline

os.chdir('..')

dp = DataPipeline()
df = dp.runPipeline(normalizeAndStandardize=True)

C:\FHNW_Programmiersachen\5_Sem\immo_challenge\src\dataPipeline.py:38: DtypeWarning: Columns (3,4,5,6,11,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,45,46,47,49,50,107,110,114,115,116,119,120,121,124,125,126,128,131,132) have mixed types. Specify dtype option on import or set low_memory=False.
  self.data = pd.read_csv(filePath)
C:\FHNW_Programmiersachen\5_Sem\immo_challenge\src\dataPipeline.py:61: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ]].bfill(axis=1)['Space extracted']
C:\FHNW_Programmiersachen\5_Sem\immo_challenge\src\dataPipeline.py:66: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To o

In [3]:
from src import torchModelRun
import torch
import torch.nn as nn
import torch.nn.functional as F



class FullyConnectedModel(nn.Module):
    import torch.nn.functional as F
    def __init__(self):
        super(FullyConnectedModel, self).__init__()
        self.fc1 = nn.Linear(109, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 1)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    

models = torchModelRun.run(FullyConnectedModel, df, 3)

Epoch 1, Loss: 155.7348597628044, Train MAPE: 184.50389505827073, Test MAPE: 167.94256217634998
Epoch 2, Loss: 101.19729408161753, Train MAPE: 337.4542379553, Test MAPE: 230.37973963711912
Epoch 3, Loss: 80.20769925496951, Train MAPE: 501.98705294125375, Test MAPE: 333.8668213694488
Epoch 4, Loss: 75.79869537408376, Train MAPE: 706.670733443614, Test MAPE: 704.2004773370151
Epoch 5, Loss: 70.88791314890378, Train MAPE: 467.58272961116364, Test MAPE: 442.568129139385
Epoch 6, Loss: 65.86295672779687, Train MAPE: 538.8632627063836, Test MAPE: 374.4633936936828
Epoch 7, Loss: 50.538664928447254, Train MAPE: 193.35206386021204, Test MAPE: 179.84531689055578
Epoch 8, Loss: 77.6649203218184, Train MAPE: 708.1845845600103, Test MAPE: 1023.4720443108073
Epoch 9, Loss: 61.485677391319236, Train MAPE: 2600.879924229213, Test MAPE: 315.40630262564883
Epoch 10, Loss: 59.52022589201667, Train MAPE: 422.58447939826107, Test MAPE: 255.0069189619744
Epoch 11, Loss: 60.26153136351193, Train MAPE: 340.9

test_loss,█▂▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▄▃▃▂▁▁▁▁▁▁
test_mape,▃▆▄▃▂▂█▂▂▂▁▂▂▂▃▁▃▁▁▁▁▁▁▁▁▁▂▁▂▁▃▄▅▁▂▁▁▁▁▁
time_test,█▃█▂▁▁▁▃▁▂▄▄▁▂▁▁▁▁▁▁▄▁▁▁▁▂▃▁▁▁▄▃▅▄▂▄▂▄▂▂
time_train,▅▇▂█▂▁▁▁▁▅▄▆▁▁▁▂▄▁▅▁▃▃▅▂▂▂▃▃▅▇▅▄▄▂▂▂▅▂▂▂
train_loss,█▅▄▃▃▂▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▁▂▂
train_mape,▂▃▃▂█▂▂▂▂▁▂▂▃▃▅▁▁▁▁▂▂▂▂▁▂▂▁▁▁▁▂▁▂▃▃▂▁▁▁▁
test_loss,11.43581
test_mape,36.42518
time_test,1.05907
time_train,2.26135
train_loss,46.19493


In [23]:
df_exclude = df.drop(columns=['price_cleaned'])
datapoint = df_exclude.iloc[0].values
print(datapoint)

model = models[0]
model.eval()
model = model.to('cpu')
model(torch.tensor(datapoint, dtype=torch.float32).unsqueeze(0))

[-0.06252029 -0.16141141 -0.15081525 -0.15818127 -0.16118794  0.07330458
  0.01860683 -0.05420477 -0.15818149 -0.16125587 -0.210203   -0.1607161
 -0.10421573 -0.31844458 -0.30211263 -0.2543027  -0.26567123 -0.19613857
 -0.13621395  0.21723425 -0.16156815  0.10853035 -0.29733819 -0.26276898
 -0.23044499 -0.04922958 -0.13332496 -0.07904496 -0.16249442 -0.15641817
 -0.15930265 -0.0863231  -0.27647055 -0.12237486 -0.06379804 -0.24325078
 -0.01523485 -0.13062437  0.01332896 -0.10886941 -0.06211918 -0.35097407
 -0.12816296 -0.10699056 -0.18118609 -0.37021714 -0.36131536 -0.17176833
 -0.14970704 -0.22814913 -0.28677628 -0.30713192 -0.32764697 -0.14018795
 -0.1119663  -0.11196448 -0.15238566 -0.15870255 -0.16169256 -0.16107613
 -0.15992203 -0.05349623 -0.25859084  0.08188449 -0.15569916 -0.01211535
 -0.04884041 -0.09895917 -0.01184354 -0.00692593 -0.10208501 -0.09608129
 -0.03761226 -0.01421981 -0.00692593 -0.00692593 -0.01839305 -0.02363568
 -0.05873742 -0.09777585 -0.01209366 -0.09965065 -0.

tensor([[0.0823]], grad_fn=<AddmmBackward0>)

In [24]:
df['price_cleaned'].iloc[0]

np.float64(0.08544731460740697)